In [1]:
import sys  
sys.path.insert(0, '../../')

In [2]:
import optuna
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

import logging
import sys

In [3]:
seed = 0
n_samples = 1000
jitter = 15 * 0.01
datasets = []

n_samples_grid = [50, 100, 200, 400, 600, 800, 1000, 1200, 1400, 1600]

In [4]:
experiment = []
n_centers = 2
rock_results = []
other_results = []

In [5]:
for n in n_samples_grid:
    step = 0
    for s in range(seed, seed+10):
        #print(f'jitter: {j}, seed: {s}')
        dataset = make_moons(n_samples=n, noise=jitter, shuffle=True, random_state=s) 
        run = {}
        run['step'] = step
        run['n_samples'] = n
        run['n_centers'] = n_centers
        run['jitter'] = jitter
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        if n_centers:
            kmeans = KMeans(n_clusters=n_centers, random_state=seed).fit(data).labels_
        elif dataset_name == 'moons':
            kmeans = KMeans(n_clusters=2, random_state=seed).fit(data).labels_
            run['k'] = 2
        eps, min_pts = (0.2, 4)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = min_pts
        spectral = SpectralClustering(n_clusters=2).fit(data).labels_
        bandwidth = estimate_bandwidth(data)

        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

In [6]:
pd.DataFrame(experiment).to_csv('two_moons_analysis_samples.csv')